In [91]:
import pandas as pd
from icecream import ic
import spacy
import progressbar
from spacy.matcher import Matcher, DependencyMatcher
# !python -m spacy download en_core_web_sm
nlp=spacy.load("en_core_web_sm")


In [92]:
data=pd.read_csv('dataframe_true_final.csv')
text=data['txt']
data=data.drop(columns=['Unnamed: 0','txt','id'])
text=text.str.replace("'","",regex=False)
text=text.str.replace("]","",regex=False)
text=text.str.replace("[","",regex=False)
text=text.str.replace(",","",regex=False)
text=text.str.replace("A ","a ",regex=False)
text=text.str.replace("And","and",regex=False)

In [93]:
newdata=pd.DataFrame(index=text.index, columns=['PERSON','JOB','JOB1','NORP','GPE','ORG','TENSE'])

matcher_nom=DependencyMatcher(nlp.vocab)
pattern_nom=[[{"RIGHT_ID":"nom","RIGHT_ATTRS":{"DEP":"nsubj"}},{"LEFT_ID":"nom","REL_OP":">>","RIGHT_ID":"reste","RIGHT_ATTRS":{}}]]
matcher_nom.add("nom",pattern_nom)

matcher_job = Matcher(nlp.vocab)
pattern_job = [[{"DEP": "attr"}]]
matcher_job.add("job", pattern_job)

matcher_job1=DependencyMatcher(nlp.vocab)
pattern_job1=[[{"RIGHT_ID":"nom","RIGHT_ATTRS":{"DEP":"attr"}},{"LEFT_ID":"nom","REL_OP":">","RIGHT_ID":"reste","RIGHT_ATTRS":{"DEP":"compound"}}]]
matcher_job1.add("job1",pattern_job1)


In [95]:
wrong=set()
for i in progressbar.progressbar(range (100)):
  doc=nlp(text[i])
  A=matcher_nom(doc)
  B=matcher_job(doc)
  C=matcher_job1(doc)
  if A!=[]:   
    newdata.loc[i]['PERSON']=doc[A[0][1][1]:A[0][1][0]+1].text
  else: 
    wrong.add(i)
  if B!=[]:   
    newdata.loc[i]['JOB']=doc[B[0][1]:B[0][2]].text
  else: 
    wrong.add(i)
  if C!=[]:
    Span=sorted(C[0][1])
    Job=doc[Span[0]:Span[1]]
    if len(Job.ents)!=0:

      if Job.ents[0].label_=='NORP':
        Job=Job[1:]
    if len(Job)!=0:
      newdata.loc[i]['JOB1']=Job.text


  for col in ['NORP','GPE','ORG']:
    for ent in doc.ents:
      if ent.label_==col and newdata.iloc[i][col]!=newdata.iloc[i][col]:
        newdata.loc[i][col]=ent.text
  try:
    if doc[matcher_verb(doc)[0][1]].morph.get("Tense")[0]=='Pres':
      newdata.loc[i]['TENSE']='is'
    else:
      newdata.loc[i]['TENSE']='was'
  except:
    wrong.add(i)



for df in [newdata, text]:
  df.drop(wrong).reset_index().drop(columns=['index'])


100% (100 of 100) |######################| Elapsed Time: 0:00:01 Time:  0:00:01


In [45]:
def essai(i): #tester le nouveau dataset
  ic(text[i])
  ic(newdata.iloc[i])
  doc=nlp(text[i])
  for token in doc:
    print(token.text, token.pos_, token.dep_, token.morph)




In [113]:
pd.concat([text,newdata],axis=1).to_csv('New_dataset.csv')